The purpose of this notebook is to visualize the taxonomy of the egoexo4d dataset and convert annotations to different levels of annotations. This will lower the total number of annotations that need to be worked with.

In [14]:
import os
import json
import numpy as np
from matplotlib.pyplot import *
import cv2
import torch

CLI_OUTPUT_DIR = "/local/juro4948/data/egoexo4d/egoexo" # Replace with the full path to the --output_directory you pass to the cli
VERSION = "v1"

METADATA_PATH = os.path.join(CLI_OUTPUT_DIR, "takes.json")
ANNOTATIONS_PATH = os.path.join(CLI_OUTPUT_DIR, "annotations")

assert os.path.exists(METADATA_PATH), f"Metadata doesn't exist at {METADATA_PATH}. Is the CLI_OUTPUT_DIR right? Do you satisfy the pre-requisites?"
assert os.path.exists(os.path.join(ANNOTATIONS_PATH, "keystep_train.json")), "Annotation metadata doesn't exist. Did you download it with the CLI?"


In [15]:
RELEASE_DIR = "/local/juro4948/data/egoexo4d/egoexo"  # NOTE: changeme
assert os.path.exists(RELEASE_DIR), "change RELEASE_DIR to where you downloaded the dataset to"

egoexo = {
    "takes": os.path.join(RELEASE_DIR, "takes.json"),
    # "takes_dropped": os.path.join(RELEASE_DIR, "takes_dropped.json"),
    "captures": os.path.join(RELEASE_DIR, "captures.json"),
    "physical_setting": os.path.join(RELEASE_DIR, "physical_setting.json"),
    "participants": os.path.join(RELEASE_DIR, "participants.json"),
    "visual_objects": os.path.join(RELEASE_DIR, "visual_objects.json"),
}

TASK_ID_CAT = {
    0: "Unknown",
    1000: "Cooking",
    2000: "Health",
    4000: "Bike Repair",
    5000: "Music",
    6000: "Basketball",
    7000: "Rock Climbing",
    8000: "Soccer",
    9000: "Dance",
}

for k, v in egoexo.items():
    egoexo[k] = json.load(open(v))

takes = egoexo["takes"] 
captures = egoexo["captures"]
takes_by_uid = {x["take_uid"]: x for x in takes}

In [16]:
cooking_takes = [take for take in takes if (take['parent_task_id'] == 1000)]
len(cooking_takes)

636

In [17]:
egocentric = True
use_downscaled = False


#  TODO:  we should parse takes.json to get the ego_rgb_code for the rgb aria take. From visual inspection it seems that it's always 214-1, but we should be sure.
if egocentric == True:
    downsample_rate = 1 # this is the sampling factor - take every <rate> samples
    camera = 'aria'
    ego_rgb_code = '214-1' #
    frame_rate = 30  # the raw frame rate ->  4k@60FPS (MP4) for GoPro devices and 1404x1404@30FPS (VRS) for Aria devices (https://docs.ego-exo4d-data.org/overview/#data)

# TODO: fill in values here. Also, how to handle the 60fps vs 30fps issue? Could downsample cam by half
elif egocentric == False:
    camera = 'cam'
    ego_rgb_code = '' 
    frame_rate = 60

In [18]:
# See raw annotations in this dictionary
keystep_anns = json.load(open(os.path.join(ANNOTATIONS_PATH, "keystep_train.json")))
keystep_anns_val = json.load(open(os.path.join(ANNOTATIONS_PATH, "keystep_val.json")))
# Get taxonomy
anns = keystep_anns["annotations"]
print(f'Length of training items: {len(anns)}')
anns_test = keystep_anns_val["annotations"]

# Add anns_test to anns dictionary
anns.update(anns_test)
print(f'Length of training + val items: {len(anns)}')

Length of training items: 671
Length of training + val items: 852


In [19]:
# only get videos with keystep annotations
cooking_uids = []
for item in takes:
    # check if this take is cooking
    cat = item['parent_task_id']
    if cat != 1000:
        continue

    # check if this take has annotations
    has_annotations = False
    for check_if_annotated in anns.keys():
        if anns[check_if_annotated]['take_uid'] == item['take_uid']:
            has_annotations = True
            break
    if not has_annotations:
        continue
    
    cooking_uids.append(item['take_uid'])

print(len(cooking_uids))
#with open('egoexo4d/all_cooking_videos.txt', mode='wt', encoding='utf-8') as myfile:
#     myfile.write('\n'.join(cooking_uids))

344


In [20]:
cooking_annotations = {}
for uid in cooking_uids:
    cooking_annotations[uid] = keystep_anns['annotations'][uid]

# collect all cooking scenarios  
cooking_scenarios = set()
for uid in cooking_annotations.keys():
    cooking_scenarios.add(cooking_annotations[uid]['scenario'])

cooking_scenarios = list(cooking_scenarios)
cooking_scenarios

['Cooking Sushi Rolls',
 'Cooking Tomato & Eggs',
 'Making Chai Tea',
 'Making Sesame-Ginger Asian Salad',
 'Making Cucumber & Tomato Salad',
 'Cooking Scrambled Eggs',
 'Making Coffee latte',
 'Cooking Pasta',
 'Making Milk Tea',
 'Cooking an Omelet',
 'Cooking Noodles']

In [21]:
import os
from treelib import Node, Tree

# Create directory if it doesn't exist
directory = "Cooking Taxonomy Trees"
if not os.path.exists(directory):
    os.makedirs(directory)

# filter cooking scenarios
taxonomy_cooking = {}
for scenario in cooking_scenarios:
    taxonomy_cooking[scenario] = keystep_anns['taxonomy'][scenario] 
    # Create taxonomy tree
    tree = Tree()
    # Add nodes to the tree
    for key, value in taxonomy_cooking[scenario].items():
        parent_id = value['parent_id']
        if parent_id is None:
            tree.create_node(tag=value['name'], identifier=value['id'])
        else:
            tree.create_node(tag=value['name'], identifier=value['id'], parent=value['parent_id'])
    # Save the tree to a text file in the specified directory
    tree.save2file(os.path.join(directory, scenario + " Taxonomy Tree.txt"))

# Concatenate all files into one
with open(os.path.join(directory, "All Taxonomy Trees.txt"), "w") as outfile:
    for scenario in cooking_scenarios:
        with open(os.path.join(directory, scenario + " Taxonomy Tree.txt")) as infile:
            outfile.write(infile.read())
        outfile.write("\n")


Convert each line into its Level 2 Taxonomy

In [22]:
def process_file(file_name, updated_taxonomy, input_directory, output_directory):
    
    # Read input file
    with open(os.path.join(input_directory, file_name), 'r') as input_file:
        lines = input_file.readlines()
    
    # Process each line
    processed_lines = []
    
    for line in lines:
        line = line.strip()  # Remove leading/trailing white spaces
        # Handle special line
        if(line == "action_start"):
            #print("Appended: action_start")
            processed_lines.append(line)
        elif(line == "action_end"):
            #print("Appended: action_end")
            processed_lines.append(line)
        else: # For rest of lines
            #print("")
            #print(f"Unprocessed Line: ", line)
            # Process line from annotation format to taxonomy format
            newline = line.replace("_", " ")  # Replace underscores with spaces
            words = newline.split()  # Split the string into words
            words[0] = words[0].capitalize()  # Convert the first word to uppercase
            newline = ' '.join(words)  # Join the words back together
            #print(f"Processed Line: ", newline)
            #print("")
            #TODO: Get a better solution for handling taxonomy that share the same name but different ID
            # Check if there is more than one item with the same value['name']
            names = [value['name'] for key, value in updated_taxonomy.items()]
            # If there is more than one item with the same value['name'], do not update taxonomy
            if names.count(newline) > 1:
                processed_line = newline
                #print(f"Multiple items share same name, using :", processed_line)
            else:
                for key, value in updated_taxonomy.items():
                    if value['name'] == newline:
                        # Set processed line to desired taxonomy level
                        processed_line = value['level_1_taxonomy_name']
                        #break
            
            # Process line from taxonomy format to annotation format
            processed_line = processed_line.lower()
            processed_line = processed_line.replace(" ", "_")
            #print(f"Appended: ",processed_line)
            processed_lines.append(processed_line)
    
    # Write processed lines to output file
    output_file_path = os.path.join(output_directory, file_name)
    with open(output_file_path, 'w') as output_file:
        for processed_line in processed_lines:
            output_file.write(processed_line + '\n')


In [23]:
for uid in cooking_uids:
    annotation = keystep_anns['annotations'][uid]
    take_name = annotation['take_name']
    scenario = annotation['scenario']
    taxonomy = keystep_anns['taxonomy'][scenario]

    #print(f"Take Name: ",take_name)

    # Initialize an empty dictionary to store Level 0 Taxonomy actions
    level_0_taxonomy = {}
    # Iterate over the taxonomy dictionary
    for key, value in taxonomy.items():
        # Check if the parent_id is 0
        if value['parent_id'] == None:
            # If yes, then add this action to the dictionary
            level_0_taxonomy[key] = value

    #print(f"Level 0 Taxonomy: ",level_0_taxonomy)
    
    level_1_taxonomy = {}
    # Iterate over the taxonomy dictionary
    for key, value in taxonomy.items():
        # Check if the parent_id is equal to any parent ID in level_0_taxonomy
        if value['parent_id'] in [v['id'] for v in level_0_taxonomy.values()]:
            # If yes, then add this action to the dictionary
            level_1_taxonomy[key] = value
            # Add higher_level taxonomies to dictionary
            level_1_taxonomy[key]['level_0_taxonomy'] = value['parent_id']

            # Add level_n_taxonomy_name to dictionary
            level_1_taxonomy[key]['level_0_taxonomy_name'] = level_0_taxonomy[str(value['parent_id'])]['name']
            level_1_taxonomy[key]['level_1_taxonomy_name'] = value['name']
            # Add self as higher level level_n_taxonomy_name
            level_1_taxonomy[key]['level_2_taxonomy_name'] = value['name']

    #print(f"Level 1 Taxonomy: ",level_1_taxonomy)

    # Initialize an empty dictionary to store Level 2 Taxonomy actions
    level_2_taxonomy = {}
    # Iterate over the taxonomy dictionary
    for key, value in taxonomy.items():
        # Check if the parent_id is equal to any parent ID in level_1_taxonomy
        if value['parent_id'] in [v['id'] for v in level_1_taxonomy.values()]:
            # If yes, then add this action to the dictionary
            level_2_taxonomy[key] = value
            # Add higher_level taxonomies to dictionary
            level_2_taxonomy[key]['level_1_taxonomy'] = value['parent_id']
            # Get the parent_id of the level_1_taxonomy
            level_1_parent_id = level_1_taxonomy[str(value['parent_id'])]['parent_id']
            level_2_taxonomy[key]['level_0_taxonomy'] = level_1_parent_id

            # Add level_n_taxonomy_name to dictionary
            level_2_taxonomy[key]['level_0_taxonomy_name'] = level_0_taxonomy[str(level_1_parent_id)]['name']
            level_2_taxonomy[key]['level_1_taxonomy_name'] = level_1_taxonomy[str(value['parent_id'])]['name']
            level_2_taxonomy[key]['level_2_taxonomy_name'] = value['name']

    #print(f"Level 2 Taxonomy: ",level_2_taxonomy)

    # Initialize an empty dictionary to store Level 3 Taxonomy actions
    level_3_taxonomy = {}
    # Iterate over the taxonomy dictionary
    for key, value in taxonomy.items():
        # Check if the parent_id is equal to any parent ID in level_2_taxonomy
        if value['parent_id'] in [v['id'] for v in level_2_taxonomy.values()]:
            # If yes, then add this action to the dictionary
            level_3_taxonomy[key] = value
            # Add higher_level taxonomies to dictionary
            level_3_taxonomy[key]['level_2_taxonomy'] = value['parent_id']
            # Get the parent_id of the level_2_taxonomy
            level_2_parent_id = level_2_taxonomy[str(value['parent_id'])]['parent_id']
            level_3_taxonomy[key]['level_1_taxonomy'] = level_2_parent_id
            # Get the parent_id of the level_1_taxonomy
            level_1_parent_id = level_1_taxonomy[str(level_2_parent_id)]['parent_id']
            level_3_taxonomy[key]['level_0_taxonomy'] = level_1_parent_id

            # Add level_n_taxonomy_name to dictionary
            level_3_taxonomy[key]['level_0_taxonomy_name'] = level_0_taxonomy[str(level_1_parent_id)]['name']
            level_3_taxonomy[key]['level_1_taxonomy_name'] = level_1_taxonomy[str(level_2_parent_id)]['name']
            level_3_taxonomy[key]['level_2_taxonomy_name'] = level_2_taxonomy[str(value['parent_id'])]['name']
            level_3_taxonomy[key]['level_3_taxonomy_name'] = value['name']
            

            

    #print(f"Level 3 Taxonomy: ",level_3_taxonomy)

    # Initialize an empty dictionary to store Level 4 Taxonomy actions
    level_4_taxonomy = {}
    # Iterate over the taxonomy dictionary
    for key, value in taxonomy.items():
        # Check if the parent_id is equal to any parent ID in level_3_taxonomy
        if value['parent_id'] in [v['id'] for v in level_3_taxonomy.values()]:
            # If yes, then add this action to the dictionary
            level_4_taxonomy[key] = value
            # Add higher_level taxonomies to dictionary
            level_4_taxonomy[key]['level_3_taxonomy'] = value['parent_id']
            # Get the parent_id of the level_3_taxonomy
            level_3_parent_id = level_3_taxonomy[str(value['parent_id'])]['parent_id']
            level_4_taxonomy[key]['level_2_taxonomy'] = level_3_parent_id
            # Get the parent_id of the level_2_taxonomy
            level_2_parent_id = level_2_taxonomy[str(level_3_parent_id)]['parent_id']
            level_4_taxonomy[key]['level_1_taxonomy'] = level_2_parent_id
            # Get the parent_id of the level_1_taxonomy
            level_1_parent_id = level_1_taxonomy[str(level_2_parent_id)]['parent_id']
            level_4_taxonomy[key]['level_0_taxonomy'] = level_1_parent_id
            
            # Add level_n_taxonomy_name to dictionary
            level_4_taxonomy[key]['level_0_taxonomy_name'] = level_0_taxonomy[str(level_1_parent_id)]['name']
            level_4_taxonomy[key]['level_1_taxonomy_name'] = level_1_taxonomy[str(level_2_parent_id)]['name']
            level_4_taxonomy[key]['level_2_taxonomy_name'] = level_2_taxonomy[str(level_3_parent_id)]['name']
            level_4_taxonomy[key]['level_3_taxonomy_name'] = level_3_taxonomy[str(value['parent_id'])]['name']
            level_4_taxonomy[key]['level_4_taxonomy_name'] = value['name']


    #print(f"Level 4 Taxonomy: ",level_4_taxonomy)

    # TODO: Make sure all possible Taxonomy levels are covered in the dataset we are using, i.e. do we need a level_5_taxonomy

    # Add each level taxonomy to the updated taxonomy
    updated_taxonomy = {}

    updated_taxonomy.update(level_0_taxonomy)
    updated_taxonomy.update(level_1_taxonomy)
    updated_taxonomy.update(level_2_taxonomy)
    updated_taxonomy.update(level_3_taxonomy)
    updated_taxonomy.update(level_4_taxonomy)

    #print(updated_taxonomy)


    # Process file
    file_name = take_name +".txt"
    #print("File Name: ", file_name)
    input_directory = '/local/juro4948/data/egoexo4d/preprocessed/annotations/gravit-groundTruth/'
    output_directory = '/local/juro4948/data/egoexo4d/preprocessed/annotations/gravit-groundTruth-level1-taxonomy/'
    #process_file(file_name, updated_taxonomy, input_directory, output_directory)
    #break # Breaking after processing first file for testing


This function is used to find the number of unique classes in the annotations. This parameter can be updated in 
configs/egoexo/egoexo_ft.yaml
num_classes: []

In [24]:
import os
import glob

def count_unique_lines(directory):
    unique_lines = set()
    for filename in glob.glob(os.path.join(directory, '*')):
        with open(filename, 'r') as f:
            lines = [line.strip() for line in f]
            unique_lines.update(lines)
    return len(unique_lines)

In [25]:
directory = '/local/juro4948/data/egoexo4d/preprocessed/annotations/gravit-groundTruth-level1-taxonomy/'
print(f"The total number of classes is: {count_unique_lines(directory)}")
print(f"Update num_classes in configs/egoexo/egoexo_ft.yaml with this number")

The total number of classes is: 169
Update num_classes in configs/egoexo/egoexo_ft.yaml with this number
